[![GitHub](https://img.shields.io/badge/Github-hibana2077-blue?style=plastic-square&logo=github)](https://github.com/hibana2077)
[![Colab](https://img.shields.io/badge/Colab-Open%20in%20Colab-blue?style=plastic-square&logo=googlecolab)](https://colab.research.google.com/github/hibana2077/hibana2077/blob/master/train/train.ipynb)

In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [8]:
#using sklearn random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from talib import abstract
from ccxt import binance,bybit
import pandas as pd
import numpy as np
import os

In [2]:
os.remove("../data/BTC_USDT_15m.csv")

In [6]:
900000/2

450000.0

In [9]:
binance = binance()
symbol = 'BTC/USDT'
timeframe = '5m'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2019-10-01T00:00:00Z')
end = binance.parse8601('2022-12-31T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 450000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

In [10]:
df

,time,open,high,low,close,volume
0,2019-10-01 00:00:00,8289.97,8310.08,8238.01,8242.33,676.180919
1,2019-10-01 00:05:00,8241.55,8249.79,8230.03,8234.22,164.891499
2,2019-10-01 00:10:00,8235.52,8245.07,8206.57,8243.39,175.031226
3,2019-10-01 00:15:00,8242.94,8252.53,8235.18,8242.69,93.312935
4,2019-10-01 00:20:00,8239.17,8239.31,8210.01,8219.42,163.807780
...,...,...,...,...,...,...
340995,2023-01-01 02:50:00,16546.22,16547.24,16544.54,16546.33,309.266290
340996,2023-01-01 02:55:00,16545.77,16549.12,16545.01,16548.19,177.664020
340997,2023-01-01 03:00:00,16548.19,16548.19,16523.34,16529.36,910.837670
340998,2023-01-01 03:05:00,16529.36,16532.86,16526.00,16530.40,327.232960


In [11]:
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['EMA'] = abstract.EMA(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['AD'] = abstract.AD(df, timeperiod=14)
df['ADOSC'] = abstract.ADOSC(df, timeperiod=14)
df['ADXR'] = abstract.ADXR(df, timeperiod=14)
df['APO'] = abstract.APO(df, timeperiod=14)
df['AROONOSC'] = abstract.AROONOSC(df, timeperiod=14)
df['BOP'] = abstract.BOP(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['CMO'] = abstract.CMO(df, timeperiod=14)
df['DX'] = abstract.DX(df, timeperiod=14)
df['MOM'] = abstract.MOM(df, timeperiod=14)
df['PPO'] = abstract.PPO(df, timeperiod=14)
df['ROC'] = abstract.ROC(df, timeperiod=14)
df['ROCP'] = abstract.ROCP(df, timeperiod=14)
df['ROCR'] = abstract.ROCR(df, timeperiod=14)
df['ROCR100'] = abstract.ROCR100(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['TRIX'] = abstract.TRIX(df, timeperiod=14)
df['ULTOSC'] = abstract.ULTOSC(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['WMA'] = abstract.WMA(df, timeperiod=14)
df['HT_TRENDLINE'] = abstract.HT_TRENDLINE(df, timeperiod=14)
df['TRANGE'] = abstract.TRANGE(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()

In [12]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()

In [13]:
#正規化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']] = scaler.fit_transform(df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']])
df[['open','high','low','close','volume']] = scaler.fit_transform(df[['open','high','low','close','volume']])

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [14]:
X = df.drop(['UP','DOWN','time','CLOSE_percent'], axis=1)
#y -> UP and DOWN
y = df['UP'] #UP

In [15]:
X

,open,high,low,close,volume,MFI,RSI,ADX,CCI,ATR,...,PPO,ROC,ROCP,ROCR,ROCR100,TRIX,ULTOSC,WMA,HT_TRENDLINE,TRANGE
63,0.069941,0.066794,0.071348,0.069808,0.012331,0.661738,0.483007,0.165436,0.379781,0.010895,...,0.503713,0.382579,0.382579,0.382579,0.382579,0.598096,0.370193,0.065810,0.061314,0.003983
64,0.069707,0.066647,0.071265,0.069971,0.007189,0.602339,0.518361,0.159479,0.375967,0.010775,...,0.504393,0.384011,0.384011,0.384011,0.384011,0.595659,0.460342,0.065787,0.061316,0.003293
65,0.069866,0.066838,0.071483,0.070015,0.005445,0.599172,0.527829,0.145157,0.443669,0.010600,...,0.505099,0.382460,0.382460,0.382460,0.382460,0.593453,0.495088,0.065770,0.061316,0.003015
66,0.069931,0.066698,0.071391,0.069818,0.004796,0.541668,0.483205,0.135582,0.399074,0.010317,...,0.503023,0.379181,0.379181,0.379181,0.379181,0.591172,0.431068,0.065727,0.061299,0.002492
67,0.069734,0.066521,0.071265,0.069806,0.006702,0.450125,0.480447,0.131700,0.379396,0.009929,...,0.501570,0.377986,0.377986,0.377986,0.377986,0.588905,0.445980,0.065685,0.061281,0.001955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340994,0.195081,0.191859,0.196855,0.195275,0.016200,0.675832,0.531087,0.128412,0.444131,0.003952,...,0.506379,0.384672,0.384672,0.384672,0.384672,0.581174,0.494083,0.191896,0.187957,0.001461
340995,0.195195,0.191863,0.196954,0.195277,0.020469,0.673424,0.531854,0.115585,0.470624,0.003671,...,0.506002,0.384187,0.384187,0.384187,0.384187,0.581087,0.482614,0.191894,0.187959,0.000445
340996,0.195189,0.191892,0.196961,0.195306,0.011759,0.659282,0.545440,0.106894,0.491443,0.003463,...,0.505674,0.383715,0.383715,0.383715,0.383715,0.580995,0.509066,0.191894,0.187963,0.000677
340997,0.195226,0.191877,0.196627,0.195015,0.060284,0.508392,0.415726,0.118544,0.278598,0.004059,...,0.505376,0.381794,0.381794,0.381794,0.381794,0.580675,0.406061,0.191856,0.187962,0.004092


In [16]:
y.value_counts()

1.0    170470
0.0    170466
Name: UP, dtype: int64

In [17]:
X = X.values
y = y.values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [19]:
X_train

array([[0.27632788, 0.27324343, 0.27784235, ..., 0.27207262, 0.26401881,
        0.00713372],
       [0.55146355, 0.54811184, 0.55207444, ..., 0.55059269, 0.55051342,
        0.01876141],
       [0.8267326 , 0.8229351 , 0.82676395, ..., 0.82821602, 0.82742852,
        0.02502565],
       ...,
       [0.23931196, 0.2359925 , 0.24071207, ..., 0.23611055, 0.23174553,
        0.00519055],
       [0.57727593, 0.57396933, 0.57933603, ..., 0.57659318, 0.57578783,
        0.00423379],
       [0.30171433, 0.29868448, 0.30259586, ..., 0.29944291, 0.29626196,
        0.0149179 ]])

In [20]:
#random forest classifier with n_estimators=10 (default)
model = RandomForestClassifier(
    n_estimators=50,
    max_depth=10)



In [21]:
#train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, n_estimators=50)

In [22]:
pred = model.predict(X_test)

print(accuracy_score(y_test, pred))

0.5354436023183237


In [23]:
pred

array([0., 1., 0., ..., 1., 0., 1.])

In [24]:
X_train[0]

array([0.27632788, 0.27324343, 0.27784235, 0.27628661, 0.04427659,
       0.81135933, 0.75767514, 0.5501772 , 0.58722876, 0.04723819,
       0.26102265, 0.27028821, 0.85444184, 0.76397494, 0.46387704,
       0.39034977, 0.58184128, 0.96428571, 0.41539705, 0.75767514,
       0.78085378, 0.63713659, 0.55509546, 0.42401154, 0.42401154,
       0.42401154, 0.42401154, 0.64998448, 0.63925663, 0.27207262,
       0.26401881, 0.00713372])

In [25]:
test_ans = model.predict(X_train)
print(accuracy_score(y_train, test_ans))

0.6026742066937294


In [26]:
#confusion matrix
confusion_matrix(y_test, pred)


array([[21938, 20520],
       [19076, 23700]], dtype=int64)

In [27]:
#classification report  
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.53      0.52      0.53     42458
         1.0       0.54      0.55      0.54     42776

    accuracy                           0.54     85234
   macro avg       0.54      0.54      0.54     85234
weighted avg       0.54      0.54      0.54     85234



In [28]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc = roc_auc_score(y_test, pred)
print('AUC: %.2f' % auc)

AUC: 0.54


In [29]:
#save model
import pickle
filename = 'RFCV2.sav'
pickle.dump(model, open(filename, 'wb'))
